# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25, randomize_index=False)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.49,2.0,132.0,3.0,0.0,1.0,0.0,support,low
1,NaN,0.63,4.0,198.0,2.0,0.0,0.0,0.0,management,medium
2,NaN,0.80,4.0,257.0,2.0,0.0,0.0,0.0,support,low
3,NaN,0.91,6.0,287.0,4.0,0.0,1.0,0.0,technical,low
4,NaN,0.52,4.0,173.0,2.0,0.0,0.0,0.0,marketing,medium


In [6]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612639,0.715936,3.801641,201.035419,3.496684,0.144342,0.237470,0.021060
std,0.248398,0.171011,1.232414,49.884041,1.459990,0.351448,0.425547,0.143589
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
len(data)

14999

In [8]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [9]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [10]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [11]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [12]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

In [13]:
reload(impyte)
pattern_log = impyte.Pattern()

#### `Pattern._check_complete_row(row)`
Determines whether a row consists out of only 1s. Method for NaN summary creation

In [14]:
pd.DataFrame([1, 1, 1]).apply(pattern_log._check_complete_row)
#pattern_log._check_complete_row()

0   -1
dtype: int64

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [15]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [16]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [17]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [18]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern._is_discrete(tmpdata, unique_instances)`
Determines on dtype and by counting unique instances whether a column contains categorical or continuous values.

In [19]:
pattern_log._is_discrete(data['satisfaction_level'], unique_instances=5) # False

False

In [20]:
pattern_log._is_discrete(data['sales'], unique_instances=5) # True

True

#### `Pattern._get_discrete_and_continuous(tmpdata, unique_instances)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [21]:
pattern_log._get_discrete_and_continuous(data, unique_instances=5)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company'],
 'discrete': ['Work_accident',
  'left',
  'promotion_last_5years',
  'sales',
  'salary']}

#### `Pattern._get_missing_value_percentage(self, data, importance_filter=False)`
Shows missing value percentage based on result and actual data table.

In [22]:
pattern_log._get_missing_value_percentage(data)

,Complete,Missing,Percentage
satisfaction_level,14625,374,2.49 %
last_evaluation,14625,374,2.49 %
number_project,14625,374,2.49 %
average_montly_hours,14625,374,2.49 %
time_spend_company,14625,374,2.49 %
Work_accident,14625,374,2.49 %
left,14625,374,2.49 %
promotion_last_5years,14625,374,2.49 %
sales,14625,374,2.49 %
salary,14625,374,2.49 %


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [23]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [24]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [25]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [26]:
pattern_log.get_continuous()

['satisfaction_level', 'last_evaluation', 'average_montly_hours']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [27]:
pattern_log.get_discrete()

['number_project',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

## `Impyter`

In [28]:
reload(impyte)
imp = impyte.Impyter() # instantiate impyte class

### Load data into imputer

#### `Impyter.load_data()`
Requires a pandas DataFrame to load. Otherwise, the input is being transformed into a DataFrame. While loading the data is being copied into the object, to stay clear of consistency issues with the original data set.

In [29]:
imp.load_data(data)

As an alternative a DataFrame can be handed over while instantiating the Impyter object.

In [30]:
imp = impyte.Impyter(data)

#### `Impyter.get_data()`
Returns the loaded data for quick reference.

In [31]:
imp.get_data().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.49,2.0,132.0,3.0,0.0,1.0,0.0,support,low
1,NaN,0.63,4.0,198.0,2.0,0.0,0.0,0.0,management,medium
2,NaN,0.80,4.0,257.0,2.0,0.0,0.0,0.0,support,low
3,NaN,0.91,6.0,287.0,4.0,0.0,1.0,0.0,technical,low
4,NaN,0.52,4.0,173.0,2.0,0.0,0.0,0.0,marketing,medium


#### `Impyter.pattern()`
Leverages `Pattern._compute_pattern()` and `Pattern.get_pattern()` methods to compute and return an overview of all existant NaN patterns in the data set. The overview shows a `NaN` in the column where a data point was missing and `1` for all complete slots. On the right hand side is a count variable to indicate how often that pattern was found. The patterns are always sorted by count and it is not given, that pattern 0 is always the pattern with only complete cases.

In [32]:
reload(impyte)
imp = impyte.Impyter(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Impyter.drop_pattern()`
Drops pattern from data set.

In [33]:
imp.drop_pattern(4).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.49,2.0,132.0,3.0,0.0,1.0,0.0,support,low
1,NaN,0.63,4.0,198.0,2.0,0.0,0.0,0.0,management,medium
2,NaN,0.80,4.0,257.0,2.0,0.0,0.0,0.0,support,low
3,NaN,0.91,6.0,287.0,4.0,0.0,1.0,0.0,technical,low
4,NaN,0.52,4.0,173.0,2.0,0.0,0.0,0.0,marketing,medium


In [34]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


If inplace flag set to True, the changes happen in the data set that's being stored in the Impyte object. Otherwise, a copy without the dropped pattern will be returned and the stored data set stays intact.

In [35]:
_ = imp.drop_pattern(4, inplace=True)

In [36]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


#### `Impyter.get_missing_summary()`
Returns table with information on missing values per column and its percentage.

In [37]:
imp.get_missing_summary()

,Complete,Missing,Percentage
satisfaction_level,14251,374,2.56 %
last_evaluation,14251,374,2.56 %
number_project,14251,374,2.56 %
average_montly_hours,14251,374,2.56 %
time_spend_company,14251,374,2.56 %
Work_accident,14251,374,2.56 %
left,14251,374,2.56 %
promotion_last_5years,14251,374,2.56 %
sales,14251,374,2.56 %


Setting the importance filter flag to `True`  shows only columns that have some missing values. This is helpful for data sets with a large amount of variables and only few nan-values.

In [38]:
for pattern_no in range(1,4): #  drop patterns 1 to 3
    imp.drop_pattern(pattern_no, inplace=True)

In [39]:
imp.get_missing_summary(importance_filter=True)

,Complete,Missing,Percentage
satisfaction_level,13129,374,2.77 %
last_evaluation,13129,374,2.77 %
time_spend_company,13129,374,2.77 %
Work_accident,13129,374,2.77 %
left,13129,374,2.77 %
promotion_last_5years,13129,374,2.77 %


#### `Impyte.one_hot_encode()`
Relies on `pandas.get_dummies()` method to transform categorical values into one-hot-encoded values.

In [40]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()

In [41]:
ohe_data = imputer.one_hot_encode(data)
ohe_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_ohe_IT,sales_ohe_RandD,...,sales_ohe_hr,sales_ohe_management,sales_ohe_marketing,sales_ohe_product_mng,sales_ohe_sales,sales_ohe_support,sales_ohe_technical,salary_ohe_high,salary_ohe_low,salary_ohe_medium
0,NaN,0.49,2.0,132.0,3.0,0.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,NaN,0.63,4.0,198.0,2.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,NaN,0.80,4.0,257.0,2.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,NaN,0.91,6.0,287.0,4.0,0.0,1.0,0.0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,NaN,0.52,4.0,173.0,2.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,0,0,0,1


#### `Impyte.one_hot_decode()`
The inversion method to `Impyte.one_hot_encode()`. Transforms one-hot-encoded columns back to categorical values.

In [42]:
imputer.one_hot_decode(ohe_data).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.49,2.0,132.0,3.0,0.0,1.0,0.0,support,low
1,NaN,0.63,4.0,198.0,2.0,0.0,0.0,0.0,management,medium
2,NaN,0.80,4.0,257.0,2.0,0.0,0.0,0.0,support,low
3,NaN,0.91,6.0,287.0,4.0,0.0,1.0,0.0,technical,low
4,NaN,0.52,4.0,173.0,2.0,0.0,0.0,0.0,marketing,medium


#### `Impyter.impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation. Scoring that is being used is f1_macro score for classifiers (supporting binary and multi-class) and r2 for regression models.

In [43]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()
complete_df = imputer.impute(estimator='rf', accuracy=[.5, .7])

Accuracy Level                Classification                Regression                    
                              0.5                           0.7                           

Pattern: Label                Score                         Estimator                     
Int64Index([], dtype='int64')


### Investigate completed data set

In [110]:
imp = impyte.Impyter(complete_df)

In [111]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,13503
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,NaN,1,374


#### `Impyter.get_result()`
Returns the results data set, once impute was performed.

In [112]:
imputer.get_result().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.88,4.0,244.0,3.0,0.0,0.0,0.0,sales,low
1,NaN,0.69,2.0,146.0,3.0,0.0,0.0,0.0,support,medium
2,NaN,0.83,5.0,270.0,5.0,1.0,1.0,0.0,support,medium
3,NaN,0.69,3.0,146.0,3.0,0.0,0.0,0.0,product_mng,medium
4,NaN,0.57,3.0,174.0,2.0,0.0,0.0,0.0,sales,low


### Comparison of complete indices methods

In [113]:
%timeit len(imputer.pattern_log.get_complete_indices())

The slowest run took 5.76 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 945 µs per loop


In [114]:
%timeit len(imputer.get_complete_old())

100 loops, best of 3: 5.45 ms per loop


In [115]:
_ = imputer.drop_pattern(7, inplace=True)

In [116]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [117]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.37,0.57,2.0,158.0,3.0,0.0,1.0,0.0,technical,low
14995,0.73,0.92,6.0,189.0,3.0,1.0,0.0,0.0,sales,medium
14996,0.57,0.74,3.0,257.0,2.0,0.0,0.0,0.0,support,low
14997,0.95,0.72,4.0,235.0,2.0,0.0,0.0,0.0,support,low
14998,0.75,0.84,5.0,222.0,3.0,1.0,0.0,0.0,IT,medium


### Accessing final models

In [118]:
mdl = imputer.get_model(2) # returns model for pattern 2

In [119]:
mdl.get_feature_name()

'average_montly_hours'

In [120]:
mdl.get_model()

[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)]

### Scaling for preprocessing

In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
scaler = StandardScaler()

In [58]:
test = data.dropna()

In [59]:
len(test[test.apply(nan_checker.is_nan)])

11259

In [60]:
test.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000
mean,0.612850,0.717751,3.798650,200.868106,3.491607,0.145395,0.238387,0.021139
std,0.247658,0.171181,1.235344,50.050638,1.453549,0.352514,0.426116,0.143853
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [61]:
test_scaled = scaler.fit_transform(test[test.corr().columns])

In [62]:
test[:5]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
3740,0.85,0.49,4.0,142.0,3.0,0.0,0.0,0.0,support,medium
3741,0.77,0.56,5.0,226.0,4.0,0.0,0.0,0.0,sales,low
3742,0.92,0.68,4.0,178.0,3.0,0.0,0.0,0.0,product_mng,low
3743,0.52,0.79,5.0,265.0,3.0,1.0,0.0,0.0,support,high
3744,0.73,0.85,5.0,245.0,3.0,0.0,0.0,0.0,RandD,low


In [63]:
pd.DataFrame(test_scaled[:5])

,0,1,2,3,4,5,6,7
0,0.957614,-1.330531,0.162998,-1.176223,-0.338226,-0.412469,-0.559467,-0.146953
1,0.634573,-0.921588,0.972526,0.502152,0.349776,-0.412469,-0.559467,-0.146953
2,1.240275,-0.220544,0.162998,-0.456920,-0.338226,-0.412469,-0.559467,-0.146953
3,-0.374930,0.422081,0.972526,1.281397,-0.338226,2.424423,-0.559467,-0.146953
4,0.473052,0.772603,0.972526,0.881784,-0.338226,-0.412469,-0.559467,-0.146953


In [64]:
test_rescaled = scaler.inverse_transform(test_scaled)

In [65]:
pd.DataFrame(test_rescaled[:5])

,0,1,2,3,4,5,6,7
0,0.85,0.49,4.0,142.0,3.0,0.0,-2.775558e-17,0.0
1,0.77,0.56,5.0,226.0,4.0,0.0,-2.775558e-17,0.0
2,0.92,0.68,4.0,178.0,3.0,0.0,-2.775558e-17,0.0
3,0.52,0.79,5.0,265.0,3.0,1.0,-2.775558e-17,0.0
4,0.73,0.85,5.0,245.0,3.0,0.0,-2.775558e-17,0.0


In [66]:
reload(impyte)
testdf = pd.read_csv('data/masterdf_201710230_andirs.csv', low_memory=False)
imp = impyte.Impyter()
_ = imp.load_data(testdf)
imp.pattern()

,Unnamed: 0,Incident Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Location_y,Address,Building_Cat,...,count all complaints not corrected,count potential fire control not corrected,count fire emergency safety,count potential fire cause,count fire emergency safety not corrected,count potential fire cause not corrected,next_fire_dpt_address,next_fire_dpt_distance,next_fire_dpt_latlong,Count
0,1,NaN,1,NaN,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,170517
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24767
2,1,1,1,1,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24


In [67]:
imp.get_missing_summary()

,Complete,Missing,Percentage
Unnamed: 0,195308,0,0.00 %
next_fire_dpt_address,195308,0,0.00 %
count potential fire cause not corrected,195308,0,0.00 %
count fire emergency safety not corrected,195308,0,0.00 %
count potential fire cause,195308,0,0.00 %
count fire emergency safety,195308,0,0.00 %
count potential fire control not corrected,195308,0,0.00 %
count all complaints not corrected,195308,0,0.00 %
count all complaints,195308,0,0.00 %
count potential fire control,195308,0,0.00 %


In [68]:
imp.get_missing_summary(importance_filter=True)

,Complete,Missing,Percentage
Incident_Year,24791,170517,87.31 %
Incident Date,24791,170517,87.31 %
Incident_Cat,24767,170541,87.32 %


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [69]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

The slowest run took 7.27 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.66 µs per loop
100000 loops, best of 3: 4.95 µs per loop


### Multi-Nans
Prediction of values with multi-nan is a last resort option. This might be suitable for certain edge cases but if the score values are low it should be considered dropping the feature or the data points.

In [44]:
multi_data = data.copy()
import numpy as np
for i in range(0, 100):
    multi_data.at[i, "last_evaluation"] = np.nan
reload(impyte)
imp = impyte.Impyter(multi_data)
_ = imp.pattern()
imp.pattern_log.get_multi_nan_pattern_nos()
imp.pattern_log.get_column_name(11)
res = imp.impute(estimator="rf", multi_nans=True)

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: sales                      0.331 (f1_macro)              RandomForestClassifier          filled...
2: average_montly_hours       0.322 (r2)                    RandomForestRegressor           filled...
3: number_project             0.576 (f1_macro)              RandomForestClassifier          filled...
4: salary                     0.518 (f1_macro)              RandomForestClassifier          filled...
5: promotion_last_5years      0.755 (f1_macro)              RandomForestClassifier          filled...
6: time_spend_company         0.584 (f1_macro)              RandomForestClassifier          filled...
7: last_evaluation            0.267 (r2)                    RandomForestRegressor           filled...
8: Work_acci

In [140]:
model = imp.get_model(11)

In [141]:
model.feature_name[0]

'satisfaction_level'

In [142]:
model.estimator_name[0]

'RandomForestRegressor'

In [143]:
model.scoring[0]

'r2'

In [144]:
model.accuracy[0]

array([ 0.40949703,  0.4190606 ,  0.40747576,  0.45047185,  0.42342288])

In [145]:
model.model[0]

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [146]:
imp2 = impyte.Impyter(res)
imp2.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14999


In [147]:
imp2.get_data()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.658000,0.601000,4.0,244.0,3.0,0.0,0.0,0.0,sales,low
1,0.704000,0.469000,2.0,146.0,3.0,0.0,0.0,0.0,support,medium
2,0.396000,0.487333,5.0,270.0,5.0,1.0,1.0,0.0,support,medium
3,0.696000,0.518000,3.0,146.0,3.0,0.0,0.0,0.0,product_mng,medium
4,0.658000,0.601000,3.0,174.0,2.0,0.0,0.0,0.0,sales,low
5,0.565000,0.494000,3.0,108.0,4.0,0.0,0.0,0.0,technical,low
6,0.679000,0.623000,6.0,118.0,2.0,0.0,0.0,0.0,IT,low
7,0.425143,0.471615,2.0,128.0,3.0,0.0,1.0,0.0,sales,medium
8,0.395000,0.502500,2.0,134.0,3.0,0.0,1.0,0.0,support,low
9,0.696000,0.518000,5.0,247.0,4.0,0.0,0.0,0.0,product_mng,medium


In [148]:
imp.save_model("testmodel.pkl")

## Ensemble
Ensemble iterates over several estimators and imputes accordingly. So far it prints the results for easy comparison.

In [45]:
imp.ensemble(estimator_list=["rf", "dt"])

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: sales                      0.332 (f1_macro)              RandomForestClassifier          filled...
2: average_montly_hours       0.324 (r2)                    RandomForestRegressor           filled...
3: number_project             0.571 (f1_macro)              RandomForestClassifier          filled...
4: salary                     0.521 (f1_macro)              RandomForestClassifier          filled...
5: promotion_last_5years      0.741 (f1_macro)              RandomForestClassifier          filled...
6: time_spend_company         0.578 (f1_macro)              RandomForestClassifier          filled...
7: last_evaluation            0.274 (r2)                    RandomForestRegressor           filled...
8: Work_acci